# Dinosaur Name Generator

In this project, we constuct an one to sequence model that inputs a starting charactor or a short word and outputs a dinosaur name. The model consists of two LSTM layers and one fully connected layer. The output layer has softmax activation with 28 classes, which are a-z characters and "\n". 

## Importing libraries and data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta0


In [3]:
!pip install tensorflow==2.0.0-beta0

     |████████████████████████████████| 87.9MB 194kB/s 
     |████████████████████████████████| 3.1MB 46.5MB/s 
     |████████████████████████████████| 501kB 45.6MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import pandas as pd
import numpy as np
import os
import csv
path = "/content/drive/My Drive/Dinosaur_name/"

In [0]:
dinosaur_name = []
with open(path+'dinos.txt','r') as textfile:
  reader = csv.reader(textfile)
  for row in reader:
    dinosaur_name.append(row[0])

In [5]:
len(dinosaur_name)

1536

There are 1536 dinosaur names in the 'dinos.txt' file. Next, "\n" will be added to all dinosaur name to indicate the end of name.



In [0]:
dinosaur_name_with_n = []
for name in dinosaur_name:
  name = name+"\n"
  dinosaur_name_with_n.append(name)

## Data preprocessing

### Importing Tokenizer
This tokenizer will token at the character level of dinosaur names.

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
tokenizer = Tokenizer(char_level = True)

Fitting tokenizer on dinosaur names.

In [0]:
tokenizer.fit_on_texts(dinosaur_name_with_n)

Creating a dictionary that transforms a letter into a number.

In [0]:
char_index = tokenizer.word_index

Creating a reverese dictionary that transforms a number into a letter.

In [0]:
index_char = {index:char for (char,index) in char_index.items()}

In [12]:
index_char

{1: 'a',
 2: 's',
 3: 'u',
 4: 'o',
 5: 'r',
 6: '\n',
 7: 'n',
 8: 'i',
 9: 'e',
 10: 't',
 11: 'l',
 12: 'p',
 13: 'h',
 14: 'c',
 15: 'g',
 16: 'd',
 17: 'm',
 18: 'y',
 19: 'b',
 20: 'k',
 21: 'v',
 22: 'x',
 23: 'z',
 24: 'j',
 25: 'w',
 26: 'f',
 27: 'q'}

### Creating n-grams
Creating n-grams from dinosaur name. These n-grams will be inputs of the model.

In [0]:
input_sequences = []
for name in dinosaur_name_with_n:
  token_list = tokenizer.texts_to_sequences([name])[0]
  
  for i in range(1,len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

Finding the maximum name length.

In [0]:
max_name_length = max([len(x) for x in input_sequences])

In [15]:
max_name_length

27

These are examples of n-grams



In [16]:
input_sequences[:13]

[[1, 1],
 [1, 1, 14],
 [1, 1, 14, 13],
 [1, 1, 14, 13, 9],
 [1, 1, 14, 13, 9, 7],
 [1, 1, 14, 13, 9, 7, 4],
 [1, 1, 14, 13, 9, 7, 4, 2],
 [1, 1, 14, 13, 9, 7, 4, 2, 1],
 [1, 1, 14, 13, 9, 7, 4, 2, 1, 3],
 [1, 1, 14, 13, 9, 7, 4, 2, 1, 3, 5],
 [1, 1, 14, 13, 9, 7, 4, 2, 1, 3, 5, 3],
 [1, 1, 14, 13, 9, 7, 4, 2, 1, 3, 5, 3, 2],
 [1, 1, 14, 13, 9, 7, 4, 2, 1, 3, 5, 3, 2, 6]]

### Padding n-grams
Padding  all n-grams to the max_name_length, zeros will be added in front of n-grams.

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_name_length,padding = 'pre'))

In [19]:
input_sequences[:13]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  1,  1, 14],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  1, 14, 13],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1, 14, 13,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  1,  1, 14, 13,  9,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  1, 14, 13,  9,  7,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1, 14, 13,  9,  7,  4,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  1, 14, 13,  9,  7,  4,  2,  1],


### Dividing inputs and labels
In this part, the padded sequence will be divided into two parts, the input and label. For example, if the sequence is

$[0,0,0,0,0,0,0,0,4,7,8,9,12,23,15]$,

the input will be all the elements except the last one, which is

$[0,0,0,0,0,0,0,0,4,7,8,9,12,23]$,

and the label will be the last element, which is

$[15]$.

In other word, the input sequence will be fed into RNN and predicts the label.


In [0]:
Xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

In [0]:
Xs = Xs.reshape(Xs.shape[0],Xs.shape[1],1)

In [23]:
Xs.shape

(18374, 26, 1)

Creating one-hot labels from the labels.

In [0]:
Ys = tf.keras.utils.to_categorical(labels)

In [25]:
Ys.shape

(18374, 28)

Now, we have Xs, which are inputs of the model, and Ys, which are one-hot label of the model. Xs will be fed into the RNN and the predicted labels will be compared with Ys.

## Creating Model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

The RNN model consists of 4 layers. The fist two layers are LSTM with 64 units each layer. The next layer is fully connected layer with 64 units. The last layer is softmax layer with 28 classes.

In [0]:
model = Sequential()
model.add(LSTM(64, input_shape = (Xs.shape[1],1), return_sequences=True, name = 'LSTM_1'))
model.add(LSTM(64, name = "LSTM_2"))
model.add(Dense(64, activation = 'relu', name = "FC"))
model.add(Dense(Ys.shape[1],activation = 'softmax', name = "OUTPUT"))

In [27]:
from tensorflow.keras.models import load_model
model = load_model(os.path.join(path,'model32_03.h5'))

W0716 12:58:40.401199 140225687484288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_1 (LSTM)                (None, 26, 64)            16896     
_________________________________________________________________
LSTM_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
FC (Dense)                   (None, 64)                4160      
_________________________________________________________________
OUTPUT (Dense)               (None, 28)                1820      
Total params: 55,900
Trainable params: 55,900
Non-trainable params: 0
_________________________________________________________________


In [0]:
from tensorflow.keras.optimizers import Adam
adam = Adam(lr = 0.00005)

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer = adam,
             metrics=['accuracy'])

In [35]:
history = model.fit(Xs,Ys, epochs = 100)

Train on 18374 samples
Epoch 1/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4754 - accuracy: 0.8290
Epoch 2/100
18374/18374 [==============================] - 21s 1ms/sample - loss: 0.4753 - accuracy: 0.8286
Epoch 3/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4752 - accuracy: 0.8292
Epoch 4/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4752 - accuracy: 0.8292
Epoch 5/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4750 - accuracy: 0.8295
Epoch 6/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4749 - accuracy: 0.8286
Epoch 7/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4746 - accuracy: 0.8292
Epoch 8/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4746 - accuracy: 0.8305
Epoch 9/100
18374/18374 [==============================] - 20s 1ms/sample - loss: 0.4745 - accuracy: 0.8290
Epoch

In [0]:
model.save(os.path.join(path,'model32_04.h5'))

## Name generator
In this section, we write a function that generates a dinosaur name. The input of the function will be a starting word, a letter or a blank. The output will be the dinosaur name.

In [0]:
def get_dinosaurname(starting):
  seed_text = starting
   
  while True:
    token_list = tokenizer.texts_to_sequences([seed_text])
    X = np.array(pad_sequences(token_list,maxlen=max_name_length-1,padding = 'pre', truncating='pre'))
    
    X=X.reshape(X.shape[0],X.shape[1],1)
    prob = model.predict_proba(X)
    idx = np.random.choice(Ys.shape[1], p=prob[0,:])
    
    output_char = index_char[idx]

    seed_text+=output_char
    
    if(output_char == '\n'):
      break
   
  return seed_text
  

In [39]:
dinosaur1 = get_dinosaurname('ty')
print(dinosaur1)

tycocephale



In [40]:
for i in range(10):
  print(get_dinosaurname(''))

venceoceratops

venosaurus

liporosaurus

vhaan

brevhongosaurus

vixianycus

moshisaurus

bahsaurus

bubazasaurus

bagaraatan

